This notebook may take some time to finish running, depending on the setttings. 

At 4k and with `fibonacci_steps = [1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 199]`

A 2015 12" MacBook 1.1 (Turbo 1.9) GHz Intel Core M processor and 8 GB 1600 MHz DDR3 RAM required ~3 hours to run this notebook. 

The running notebook maintained ~20 GB RAM usage (thank you swap file and fast storage!) and commanded the majority of the CPU while running.

2675 individual png images are generated which total ~5 GB storage. 




> For the shorter animation with`fibonacci_steps = [199]` this notebook only takes about 15 minutes to finish.


# Requirements:

>`python > 3.7` (not tested in python < 3.7

>`matplotlib > 3`

>`numpy > 15`

>`colorcet` https://github.com/pyviz/colorcet

> `ffmpeg`


In [1]:
import sys
import colorcet as cc
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

from matplotlib import cm
from typing import Dict, Tuple, List, Iterator

%config InlineBackend.figure_format = 'retina'
np.warnings.filterwarnings("ignore")


In [2]:
def get_bounds_for_coordinate_and_width(
    x: float = -1.0, y: float = 0.0, width=2.5
) -> Tuple[Tuple[float], Tuple[float]]:
    """Provides a square window around a point (x, y)
    """

    x_bounds = x - width / 2, x + width / 2
    y_bounds = y - width / 2, y + width / 2

    return x_bounds, y_bounds


In [3]:
def mandlebrot_set_generator(
    real_bounds: Tuple[float] = (-2.0, 0.5),
    imag_bounds: Tuple[float] = (-1.5, 1.5),
    real_count: int = 1000,
    imag_count: int = 1000,
    z_max: int = 50,
    iterations_max: int = 200,
    result_only: bool = True,
    verbose: bool = True,
    comparison: str = 'real',
) -> np.ndarray:
    """Compute the mandlebrot set Z = Z*Z + C
    
    Parameters
    ----------
    real_bounds : Tuple[float], optional
        Real min, real max.
    imag_bounds : Tuple[float], optional
        Imaginary min, imaginary max.
    real_count : int, optional
        Number of real values.
    imag_count : int, optional
        Number of imaginary values.
    z_max : int, optional
        Maximum allowed z value. Values above this are considered to have blown up.
    iterations_max : int, optional
        Number of iterations to compute. Also used in computations.
    result_only :bool, optional
        True returns only the last iteration, False returns all iterations
    comparison :str, optional
        'real', 'imag', 'norm' - what each points value is taken to be (real component, imag, or the euclidean norm)
        imag/real adds 'veins' to the leafs, 'norm' merges leafs
    Returns
    -------
    np.ndarray
        if result_only: 
            2D array of the last iteration
        else:
            3D array with all iterations
    """

    image = np.ones((imag_count, real_count)) * iterations_max

    if not result_only:
        image_stack = np.zeros(
            (iterations_max, imag_count, real_count), dtype=np.float128
        )

    reals = np.linspace(*real_bounds, real_count)
    imags = np.linspace(*imag_bounds, imag_count)

    R, I = np.meshgrid(reals, imags)
    C = R + 1j * I

    Z = np.zeros((imag_count, real_count), dtype=np.complex256)

    for iteration in range(1, iterations_max):
        if verbose:
            sys.stdout.write("\r{:0{:}d}/{}".format(iteration + 1, len(str(abs(iterations_max))), iterations_max))

        if comparison == 'real':
            good_vals = Z.imag <= z_max
            
        elif comparison == 'imag':
            good_vals = Z.real <= z_max
            
        elif comparison == 'norm':
            good_vals = np.abs(Z) <= z_max
            
        Z[good_vals] = C[good_vals] + Z[good_vals] * Z[good_vals]
        image[good_vals] = iteration

        if not result_only:
            image_stack[iteration] = image

    if result_only:
        return image
    else:
        return image_stack


In [4]:
def image_to_image_stack_by_values(
    image: np.ndarray, start: int, stop: int, delta: int, clip: int
) -> np.ndarray:
    """Break a single image into many based on contour values.
    TODO: Come up with a better description... 
    
    Args:
        image (np.ndarray): Description
        start (int): Description
        stop (int): Description
        delta (int): Description
        clip (int): Description
    """
    image_stack = []
#     for i in range(start, stop - delta):
    for i in range(start - delta, stop):


        low = i
        high = low + delta

        modified = np.copy(image)
        modified[modified > high] = clip
        modified[modified <= low] = clip
        image_stack.append(modified)

    return image_stack


In [5]:
def figure_without_frame(
    width: float = 5, height: float = 5, dpi: int = 300
) -> Tuple[matplotlib.figure.Figure, matplotlib.axes.SubplotBase]:
    """Setup a figure with no frame, axes, or ticks.
    
    Parameters
    ----------
    width : float, optional
        Figure width in inches.
    height : float, optional
        Figure height in inches.
    dpi : int, optional
        Pixels per inch
    
    Returns
    -------
    matplotlib.figure.Figure, matplotlib.axes.SubplotBase
        fig, ax
    """
    fig = plt.figure(frameon=False, dpi=dpi)
    fig.set_size_inches(w=width, h=height)

    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()

    fig.add_axes(ax)

    return fig, ax


In [6]:
def render_image_by_contour_steps(
    image: np.ndarray,
    contour_steps: List[int],
    directory: str,
    file_prefix: str,
    extent: Tuple[int, int, int, int],
    height: float,
    width: float,
    dpi: int,
    verbose: bool = True,
) -> None:
    """Given an image, break it into contours given by contour_steps and render each as a png"""

    for delta in contour_steps:

        stack = image_to_image_stack_by_values(
            image, int(np.min(image)), int(np.max(image)), delta, int(np.max(image))
        )

        fig, ax = figure_without_frame(width=width, height=height, dpi=dpi)
        norm = cm.colors.Normalize(vmin=np.min(image), vmax=np.max(image) + 1)

        imshow = ax.imshow(stack[0], cmap=cc.m_fire, norm=norm, extent=extent)

        for k in range(len(stack)):
            number_of_digits = len(str(len(stack)))
            file_number = "{:0{:}d}_{:0{:}d}".format(
                delta, number_of_digits, k, number_of_digits
            )

            if verbose:

                sys.stdout.write("\r" + file_number)

            imshow.set_data(stack[k])
            fig.savefig(
                directory + "/" + file_prefix + "_" + file_number + ".png", dpi="figure"
            )
    plt.close(fig)
            
    return None


In [7]:
def fibonacci_generator(
    a: int = -1, b: int = 1, stop: int = 200
) -> Iterator[int]:

    while a + b < stop:
        a, b = b, a + b

        yield b


In [8]:
real_bounds = -0.7794302918017602 + 0.00054, -0.7891570171375637- 0.00059 # adjustment to get to 4k resolution

imag_bounds = 0.14489532190829088, 0.151
imag_count = 2160

height_to_width_ratio = abs(np.diff(real_bounds) / np.diff(imag_bounds) )[0]

# to make the sure the pixels are square
real_count = int(imag_count * height_to_width_ratio)

# make sure its an even number
while real_count % 10 != 0:
    real_count -= 1
real_count

3840

In [13]:
iterations_max = 200

In [14]:
interesting_set = mandlebrot_set_generator(real_bounds=real_bounds, imag_bounds=imag_bounds, real_count=real_count, imag_count=imag_count, iterations_max=iterations_max)

200/200

In [15]:
fibonacci_steps = [i for i in fibonacci_generator(0,1)]
fibonacci_steps

[1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144]

In [16]:
fibonacci_steps.append(iterations_max - 1)
fibonacci_steps

[1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 199]

# NOTE:
the above set of steps create a movie that is extremely long. The list below will generate a far shorter animation. 

Comment out the cell below to generate the full animation.

In [1]:
fibonacci_steps = [199]

In [20]:
extent = [*real_bounds, *imag_bounds]

In [22]:
dpi = 300
height = imag_count / dpi 
width = real_count / dpi
height, width

(7.2, 12.8)

In [23]:
render_image_by_contour_steps(
    image=interesting_set,
    contour_steps=fibonacci_steps,
    directory="project_animation_fire_v3",
    file_prefix="mandelbrot",
    extent=extent,
    width=width,
    height=height, 
    dpi=dpi
)

199_373

# Generate the animation:

In terminal navigate to the directory with the images and run this single line command:

**For the long `fibonacci_steps = [1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 199]` animation, at 4k**

`ffmpeg -framerate 60 -pattern_type glob -i "mandelbrot_*.png" -s:v 3840x2160 -c:v libx264 -preset slow -profile:v high -crf 17 -coder 1 -pix_fmt yuv420p -movflags +faststart animation.mp4`

**For the short `fibonacci_steps = [199]` animation, at 1080p**

`ffmpeg -framerate 60 -pattern_type glob -i "mandelbrot_*.png" -s:v 1920x1080 -c:v libx264 -preset slow -profile:v high -crf 17 -coder 1 -pix_fmt yuv420p -movflags +faststart animation1080p.mp4`

## To generate the gif:

**first generate a pallete**

`ffmpeg -y -i animation1080p.mp4 -vf fps=60,scale=1920:-1:flags=lanczos,palettegen palette1080.png`

**then use the pallete to generate the gif**

`ffmpeg -i animation1080p.mp4 -i palette1080.png -filter_complex "fps=30,scale=1920:-1:flags=lanczos[x];[x][1:v]paletteuse" output1080_30.gif`

**ffmpeg notes**

`-framerate 60 -pattern_type glob -i "mandelbrot_*.png"` set the input to be the files in the current directory that with filenames that start with `mandelbrot_` and end with `.png`, use `glob` to collect all those filenames and place thin alphanumeric order, nd take the input to be 60 frames per second.

`-s:v 3840x2160` set the frame size of the stream

`-c:v libx264` set the codec of the video stream to be `libx264`

`-preset slow` encoding speed vs compression ratio.  `faster,fast, medium, slow, slower, veryslow` more time makes for better compression.

`-profile:v high` H.264 profile setting = `high`

`-crf 17` Constant Rate Factor `0 - 51`, exponential, `0` = lossless encoding, `17 - 28` is "acceptable"

`-coder 1` ???

`-movflags +faststart` If video to be viewed in a browser (allows playing before video is fully downloaded).